In [ ]:
import mqr
from mqr.plot import Figure

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from IPython.display import display

# Datasets

In [ ]:
# Silicon wafer resistivity - NIST 
# F 84 - 99

columns = ['RUNID', 'WAFERID', 'PROBE', 'MONTH', 'DAY', 'OPERATOR', 'TEMP', 'AVERAGE', 'STDDEV',]
dtype = {
    'WAFERID': int,
    'PROBE':int,
}

data = pd.read_csv(
    'https://www.itl.nist.gov/div898/software/dataplot/data/MPC61.DAT',
    skiprows=50,
    header=None,
    names=columns,
    sep='\\s+',
    dtype=dtype,
)
data['REPEAT'] = np.repeat([1,2,3,4,5,6,7,8,9,10,11,12], 25)
data

---
# GRR

In [ ]:
# Use tolerance of 8 Ohm cm, and test for variance contribution from the probe (listed as operator)
tol = 8.0
names = mqr.msa.NameMapping(part='WAFERID', operator='PROBE', replicate='REPEAT', measurement='AVERAGE')
names

In [ ]:
# Use only RUNID==1, as in the lecture notes
grr = mqr.msa.GRR(data[data['RUNID']==1], tol, names=names, include_interaction=True)

In [ ]:
mqr.nbtools.vstack(
    grr.adequacy_table,
    grr.anova_table,
    grr.grr_table)

NB: one outlier skews the residuals. It is visible at the very low quantiles in the probability plot.
* The outlier is low enough to reject the JB null-hyp with significance >95%.
* The Anderson-Darling test doesn't find enough evidence to reject the null-hyp that the residuals are normal.

Consider reviewing the test conditions for that datapoint to ensure the data is likely from the same process as the others.

In [ ]:
with Figure(7, 4, 2, 2, height_ratios=[2, 1]) as (fig, ax):
    mqr.plot.regression.residuals(grr.regression_result, tr='studentised', axs=ax)
    plot = mqr.nbtools.grab_figure(fig)

mqr.nbtools.hstack(
    plot,
    mqr.nbtools.vstack(
        grr.adequacy_table,
        mqr.inference.dist.test_1sample(grr.residuals, test='ad-norm')))

---
# Models

In [ ]:
mqr.anova.groups(
    grr.data,
    value=names.measurement,
    factor=names.operator)

In [ ]:
mqr.anova.interactions(
    grr.data,
    value=names.measurement,
    between=(names.part, names.operator))

---
## Plots

In [ ]:
with Figure(12, 6, 3, 2) as (fig, ax):
    mqr.plot.msa.grr(grr, axs=ax)